## 1. Import libraries

In [47]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import random
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt

## 2. Import data

In [48]:
# Import data
df_genome_scores = pd.read_csv('C:/Users/daanv/Custom Datascience Projects/HU/Movielens/Data/genome_scores.csv')
df_genome_tags = pd.read_csv('C:/Users/daanv/Custom Datascience Projects/HU/Movielens/Data/genome_tags.csv')
# df_link = pd.read_csv('C:\Users\daanv\Custom Datascience Projects\HU\Movielens\Data\link.csv')
df_movies = pd.read_csv('C:/Users/daanv/Custom Datascience Projects/HU/Movielens/Data/movie.csv')
df_ratings = pd.read_csv('C:/Users/daanv/Custom Datascience Projects/HU/Movielens/Data/rating.csv')
df_tags = pd.read_csv('C:/Users/daanv/Custom Datascience Projects/HU/Movielens/Data/tag.csv')

## 3. Check different datasets

In [49]:
df_genome_scores

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
11709763,131170,1124,0.58775
11709764,131170,1125,0.01075
11709765,131170,1126,0.01575
11709766,131170,1127,0.11450


In [50]:
df_genome_tags

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [51]:
df_ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [52]:
df_tags

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18
...,...,...,...,...
465559,138446,55999,dragged,2013-01-23 23:29:32
465560,138446,55999,Jason Bateman,2013-01-23 23:29:38
465561,138446,55999,quirky,2013-01-23 23:29:38
465562,138446,55999,sad,2013-01-23 23:29:32


In [53]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


## 4. Reduce dataframe rows

In [54]:
df_genome_scores.describe()

,movieId,tagId,relevance
count,1.170977e+07,1.170977e+07,1.170977e+07
mean,2.584297e+04,5.645000e+02,1.164833e-01
std,3.467615e+04,3.256254e+02,1.542463e-01
min,1.000000e+00,1.000000e+00,2.500000e-04
25%,2.926000e+03,2.827500e+02,2.425000e-02
50%,6.017000e+03,5.645000e+02,5.650000e-02
75%,4.606200e+04,8.462500e+02,1.415000e-01
max,1.311700e+05,1.128000e+03,1.000000e+00


In [55]:
df_genome_scores = df_genome_scores[df_genome_scores['relevance'] > 0.05]

In [56]:
df_genome_scores

,movieId,tagId,relevance
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
5,1,6,0.21700
6,1,7,0.06700
...,...,...,...
11709753,131170,1114,0.15375
11709755,131170,1116,0.31125
11709762,131170,1123,0.64375
11709763,131170,1124,0.58775


In [57]:
df_genome_scores = df_genome_scores.sample(frac=0.1, random_state=42)

In [58]:
df_genome_scores

,movieId,tagId,relevance
5098485,5103,1054,0.32375
4440309,4452,502,0.06475
11576306,117109,771,0.14975
11273957,107141,726,0.28000
914507,910,828,0.18250
...,...,...,...
1856470,1907,911,0.21350
8714852,44731,1053,0.06300
4689828,4692,733,0.48475
4354528,4366,449,0.08375


In [59]:
df_ratings.describe()

,userId,movieId,rating
count,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904587e+04,9.041567e+03,3.525529e+00
std,4.003863e+04,1.978948e+04,1.051989e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,3.439500e+04,9.020000e+02,3.000000e+00
50%,6.914100e+04,2.167000e+03,3.500000e+00
75%,1.036370e+05,4.770000e+03,4.000000e+00
max,1.384930e+05,1.312620e+05,5.000000e+00


In [60]:
# Randomly sample 10% of the rows
df_ratings = df_ratings.sample(frac=0.1, random_state=42)

In [61]:
df_ratings

,userId,movieId,rating,timestamp
17679788,122270,8360,3.5,2012-04-22 01:07:04
7106385,49018,32,2.0,2001-09-11 07:50:36
12970708,89527,109374,3.5,2015-01-06 09:26:40
15426752,106704,1060,3.0,2000-01-22 21:27:57
6934678,47791,1732,2.0,2006-01-19 15:48:23
...,...,...,...,...
4463605,30510,2628,2.0,2010-12-15 23:16:40
1798230,12140,33838,4.5,2005-11-06 06:58:50
16323283,112993,170,4.0,1996-05-12 15:39:44
4307362,29335,3616,2.5,2011-11-23 13:18:27


## 5. Combine datasets

In [62]:
# Combine genome_scores and genome_tags using tagId to get the tag descriptions.
genome = pd.merge(df_genome_scores, df_genome_tags, on="tagId")
genome.head()

,movieId,tagId,relevance,tag
0,5103,1054,0.32375,underrated
1,73023,1054,0.24825,underrated
2,53161,1054,0.34775,underrated
3,71484,1054,0.18000,underrated
4,2812,1054,0.23700,underrated


In [63]:
# Combine the genome DataFrame with movies using movieId to add movie titles
genome_with_movies = pd.merge(genome, df_movies, on="movieId")
genome_with_movies.head()

,movieId,tagId,relevance,tag,title,genres
0,5103,1054,0.32375,underrated,"Sandlot, The (1993)",Children|Comedy|Drama
1,5103,412,0.83075,friendship,"Sandlot, The (1993)",Children|Comedy|Drama
2,5103,1024,0.18175,thought-provoking,"Sandlot, The (1993)",Children|Comedy|Drama
3,5103,513,0.17450,holiday,"Sandlot, The (1993)",Children|Comedy|Drama
4,5103,731,0.12925,nudity (topless - notable),"Sandlot, The (1993)",Children|Comedy|Drama


In [64]:
# Merge ratings with movies using movieId to bring in movie titles.
ratings_with_movies = pd.merge(df_ratings, df_movies, on="movieId")
ratings_with_movies

,userId,movieId,rating,timestamp,title,genres
0,122270,8360,3.5,2012-04-22 01:07:04,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Romance
1,45237,8360,4.0,2009-06-14 13:00:51,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Romance
2,108979,8360,4.0,2011-07-24 21:16:10,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Romance
3,52095,8360,4.0,2004-07-15 09:34:54,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Romance
4,45533,8360,5.0,2007-10-24 23:24:51,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Romance
...,...,...,...,...,...,...
2000021,70201,27840,3.5,2009-11-07 22:21:51,"Day Without a Mexican, A (2004)",Comedy|Drama|Fantasy|Mystery
2000022,48527,78066,3.0,2011-11-05 23:06:38,Sada (1998),Drama
2000023,104916,100013,3.0,2013-09-30 19:53:13,Jason Becker: Not Dead Yet (2012),Documentary
2000024,57050,69222,2.5,2012-09-02 08:53:18,Perched on a Tree (Sur un arbre perché) (1971),Comedy


In [65]:
ratings_with_movies = ratings_with_movies.sample(frac=0.3, random_state=42)
ratings_with_movies

,userId,movieId,rating,timestamp,title,genres
1600826,3318,782,2.0,2000-11-28 02:31:26,"Fan, The (1996)",Drama|Thriller
463596,32908,902,4.0,2011-04-19 23:45:38,Breakfast at Tiffany's (1961),Drama|Romance
1583027,48465,333,2.0,2007-03-19 18:48:44,Tommy Boy (1995),Comedy
1374036,74633,5673,5.0,2003-10-13 17:18:57,Punch-Drunk Love (2002),Comedy|Drama|Romance
1450725,83819,42163,3.0,2013-10-27 20:23:09,Richard Pryor: Live in Concert (1979),Comedy|Documentary
...,...,...,...,...,...,...
326287,42991,4011,4.5,2008-10-29 06:42:02,Snatch (2000),Comedy|Crime|Thriller
1888645,87992,81537,3.5,2010-11-26 23:03:36,Due Date (2010),Comedy
1625432,75712,798,3.0,1997-05-25 22:01:22,Daylight (1996),Action|Adventure|Drama|Thriller
1908877,69082,2368,3.0,1998-11-18 13:51:13,King Kong Lives (1986),Adventure|Sci-Fi


In [66]:
# Combine tags and movies using movieId to link user tags with movie titles.
tags_with_movies = pd.merge(df_tags, df_movies, on="movieId")
tags_with_movies

,userId,movieId,tag,timestamp,title,genres
0,18,4141,Mark Waters,2009-04-24 18:19:40,Head Over Heels (2001),Comedy|Romance
1,66193,4141,naive,2011-01-08 00:22:10,Head Over Heels (2001),Comedy|Romance
2,70201,4141,Mark Waters,2011-06-17 17:53:44,Head Over Heels (2001),Comedy|Romance
3,89274,4141,Freddie Prinze Jr.,2013-07-30 06:01:35,Head Over Heels (2001),Comedy|Romance
4,65,208,dark hero,2013-05-10 01:41:18,Waterworld (1995),Action|Adventure|Sci-Fi
...,...,...,...,...,...,...
465559,138301,115373,French,2014-10-22 22:09:10,"Miss and the Doctors (Tirez la langue, mademoiselle) (2013)",Drama|Romance
465560,138301,115373,Louise Bourgoin,2014-10-22 22:09:19,"Miss and the Doctors (Tirez la langue, mademoiselle) (2013)",Drama|Romance
465561,138301,115373,love triangle,2014-10-22 22:19:50,"Miss and the Doctors (Tirez la langue, mademoiselle) (2013)",Drama|Romance
465562,138301,115373,single mother,2014-10-22 22:19:22,"Miss and the Doctors (Tirez la langue, mademoiselle) (2013)",Drama|Romance


In [67]:
# tags_with_movies = tags_with_movies.sample(frac=0.3, random_state=42)
# tags_with_movies

In [68]:
# Start with a smaller merge
partial_merge = pd.merge(genome_with_movies, ratings_with_movies, on="movieId", how="inner")

# Then merge the result with the tags
full_dataset = pd.merge(partial_merge, tags_with_movies, on=["movieId", "userId"], how="inner")

In [69]:
full_dataset

,movieId,tagId,relevance,tag_x,title_x,genres_x,userId,rating,timestamp_x,title_y,genres_y,tag_y,timestamp_y,title,genres
0,5103,1054,0.32375,underrated,"Sandlot, The (1993)",Children|Comedy|Drama,31273,5.0,2008-10-06 03:12:02,"Sandlot, The (1993)",Children|Comedy|Drama,Baseball Movie,2008-10-06 14:52:53,"Sandlot, The (1993)",Children|Comedy|Drama
1,5103,412,0.83075,friendship,"Sandlot, The (1993)",Children|Comedy|Drama,31273,5.0,2008-10-06 03:12:02,"Sandlot, The (1993)",Children|Comedy|Drama,Baseball Movie,2008-10-06 14:52:53,"Sandlot, The (1993)",Children|Comedy|Drama
2,5103,1024,0.18175,thought-provoking,"Sandlot, The (1993)",Children|Comedy|Drama,31273,5.0,2008-10-06 03:12:02,"Sandlot, The (1993)",Children|Comedy|Drama,Baseball Movie,2008-10-06 14:52:53,"Sandlot, The (1993)",Children|Comedy|Drama
3,5103,513,0.17450,holiday,"Sandlot, The (1993)",Children|Comedy|Drama,31273,5.0,2008-10-06 03:12:02,"Sandlot, The (1993)",Children|Comedy|Drama,Baseball Movie,2008-10-06 14:52:53,"Sandlot, The (1993)",Children|Comedy|Drama
4,5103,731,0.12925,nudity (topless - notable),"Sandlot, The (1993)",Children|Comedy|Drama,31273,5.0,2008-10-06 03:12:02,"Sandlot, The (1993)",Children|Comedy|Drama,Baseball Movie,2008-10-06 14:52:53,"Sandlot, The (1993)",Children|Comedy|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743950,104906,474,0.25200,grindhouse,Austenland (2013),Comedy|Romance,3029,3.5,2014-06-29 14:42:51,Austenland (2013),Comedy|Romance,handsome actor,2014-06-29 14:45:20,Austenland (2013),Comedy|Romance
743951,104906,474,0.25200,grindhouse,Austenland (2013),Comedy|Romance,3029,3.5,2014-06-29 14:42:51,Austenland (2013),Comedy|Romance,Jane Austen,2014-06-29 14:44:42,Austenland (2013),Comedy|Romance
743952,104906,474,0.25200,grindhouse,Austenland (2013),Comedy|Romance,3029,3.5,2014-06-29 14:42:51,Austenland (2013),Comedy|Romance,light,2014-06-29 14:44:42,Austenland (2013),Comedy|Romance
743953,104906,474,0.25200,grindhouse,Austenland (2013),Comedy|Romance,3029,3.5,2014-06-29 14:42:51,Austenland (2013),Comedy|Romance,prejudice romance,2014-06-29 14:44:42,Austenland (2013),Comedy|Romance


## 6. Create the algorithm

In [70]:
# Collaborative Filtering - Generate a User-Movie Matrix
def generate_user_movie_matrix(full_dataset):
    user_movie_matrix = full_dataset.pivot_table(index="userId", columns="movieId", values="rating").fillna(0)
    return user_movie_matrix

In [71]:
# Find Similar Users (Collaborative Filtering)
def find_similar_users(user_movie_matrix, user_id):
    user_ratings = user_movie_matrix.loc[user_id].values.reshape(1, -1)
    similarity_scores = cosine_similarity(user_ratings, user_movie_matrix.values)[0]
    user_similarity = pd.Series(similarity_scores, index=user_movie_matrix.index).sort_values(ascending=False)
    return user_similarity

In [72]:
# Content-Based Filtering - Find Tag Relevance for Movies
def calculate_content_similarity(full_dataset, movie_ids):
    # Prepare the tag relevance data
    tag_relevance = full_dataset[["movieId", "tagId", "relevance"]]
    tag_relevance = tag_relevance.groupby("movieId")["tagId"].apply(lambda x: " ".join(map(str, x))).reset_index()

    # Vectorize tags using TF-IDF
    vectorizer = TfidfVectorizer()
    tag_vectors = vectorizer.fit_transform(tag_relevance["tagId"])

    # Calculate cosine similarity for content-based filtering
    movie_idx = tag_relevance[tag_relevance["movieId"].isin(movie_ids)].index
    similarity_scores = cosine_similarity(tag_vectors[movie_idx], tag_vectors)
    return similarity_scores

In [73]:
# Hybrid Recommendations
def recommend_movies_hybrid(full_dataset, user_id, user_movie_matrix, top_n=10):
    # Collaborative Filtering: Find similar users
    user_similarity = find_similar_users(user_movie_matrix, user_id)
    similar_users = user_similarity.index[1:11]  # Top 10 similar users (exclude the user itself)

    # Get movies watched by similar users
    watched_movies = full_dataset[full_dataset["userId"].isin(similar_users)]
    watched_movie_ids = watched_movies["movieId"].unique()

    # Content-Based Filtering: Find movies similar to those watched
    content_similarity = calculate_content_similarity(full_dataset, watched_movie_ids)

    # Combine scores from collaborative filtering and content-based filtering
    combined_scores = np.mean(content_similarity, axis=0)
    movie_scores = pd.DataFrame({
        "movieId": full_dataset["movieId"].unique(),
        "score": combined_scores
    }).sort_values(by="score", ascending=False)

    # Filter out movies already rated by the user
    user_rated_movies = full_dataset[full_dataset["userId"] == user_id]["movieId"]
    recommendations = movie_scores[~movie_scores["movieId"].isin(user_rated_movies)]

    # Add movie titles
    recommendations = pd.merge(recommendations, full_dataset[["movieId", "title"]].drop_duplicates(), on="movieId")
    return recommendations.head(top_n)

In [74]:
# Random User ID for recommendations
user_id = random.choice(full_dataset["userId"].unique())

## Users most liked movies

In [79]:
def get_top_10_rated_movies_by_user(user_id, ratings_df, movies_df):
    # Filter ratings for this user
    user_ratings = ratings_df[ratings_df['userId'] == user_id]

    if user_ratings.empty:
        print(f"No ratings found for user {user_id}")
        return pd.DataFrame()

    # Sort by highest rating, then by timestamp (optional)
    top_ratings = user_ratings.sort_values(by='rating', ascending=False).head(10)

    # Merge with movie titles
    top_movies = pd.merge(top_ratings, movies_df, on='movieId')[['movieId', 'title', 'rating']]
    return top_movies

# Example usage:
top10 = get_top_10_rated_movies_by_user(user_id, df_ratings, df_movies)
print(f"Most liked movies from user ID: {user_id}\n {top10}")


Most liked movies from user ID: 70279
    movieId                                              title  rating
0      915                                     Sabrina (1954)     5.0
1     2467  Name of the Rose, The (Name der Rose, Der) (1986)     5.0
2     7361       Eternal Sunshine of the Spotless Mind (2004)     5.0
3    45210                                   United 93 (2006)     4.5
4    38798                                In Her Shoes (2005)     4.5
5     3405                        Night to Remember, A (1958)     4.0
6     6673                Ay, Carmela! (¡Ay, Carmela!) (1990)     4.0
7      518                      Road to Wellville, The (1994)     4.0
8     3448                       Good Morning, Vietnam (1987)     4.0
9    25886                              Random Harvest (1942)     4.0


## Recommendations based on users preferences

In [76]:
# Generate user-movie matrix
user_movie_matrix = generate_user_movie_matrix(full_dataset)

# Get top 10 movie recommendations for the user
recommendations = recommend_movies_hybrid(full_dataset, user_id, user_movie_matrix, top_n=10)

print(f"Recommendation based on userId: {user_id}\n {recommendations}")

Recommendation based on userId: 70279
    movieId     score                                       title
0     2987  0.123200             Who Framed Roger Rabbit? (1988)
1   118896  0.118107                                Mommy (2014)
2    27801  0.117088  Ong-Bak: The Thai Warrior (Ong Bak) (2003)
3    92236  0.115190                          In Darkness (2011)
4     3618  0.115071                    Small Time Crooks (2000)
5    66097  0.114944                             Coraline (2009)
6    46530  0.112629                     Superman Returns (2006)
7    61075  0.111442                                Elegy (2008)
8    70293  0.110584                        Julie & Julia (2009)
9     2143  0.108659                               Legend (1985)


## Check accuracy of algorithm